In [1]:
# import required libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# i use seaborn and matplotlib only during analysis and in the final product presented to the client i use Plotly
import plotly.express as px
import dash.html as html
import dash.dcc as dcc
from dash import Dash,callback,Input,Output
import plotly.graph_objects as go


In [2]:
# import the data 
df=pd.read_csv("survey_results_public.csv")
schema=pd.read_csv("survey_results_schema.csv")

In [3]:
# set the main Setting
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_columns',None)
pd.set_option("display.max_row",10)
pd.set_option("display.min_rows",10)

In [4]:
questions_list=df.columns.to_list()
# Employment_status=["Employed, full-time","Employed, part-time","freelancer","Student","Employed,"]
color_list=['#FFDF00', '#2db923', '#083248','#d459cd', '#f088eb', '#39e641' ]

#count column content

def count_column_content(df,column_name):
    x=df[column_name].str.split(";")
    x.dropna(inplace=True)
    temp_dic={}
    for i in x:
        for j in i:
            if j not in temp_dic:
                temp_dic[j]=1
            else:
                temp_dic[j]+=1
    temp_df=pd.DataFrame(pd.Series(temp_dic))
    temp_df.rename(columns={0:"Count"},inplace=True)
    temp_df.reset_index(inplace=True)
    temp_df.rename(columns={"index":column_name},inplace=True)
    temp_df.sort_values(by="Count",inplace=True,ascending=False)
    temp_df.reset_index(inplace=True)
    temp_df.drop(columns="index",axis=1,inplace=True)
    return temp_df


def cleaning_expiernce_years_of_writing_code(x):
    if isinstance(x,str):
        if x=='More than 50 years':
            x=51
            return x
        elif x=='Less than 1 year':
            x=0
            return x        
        else:
            x=x
            return(x)
    else:
        return x

# getting the upper and lower limit
def get_upper_lower_limit(data,x):
    DataWithoutNan=data[x].dropna()
    q1 = np.percentile(DataWithoutNan,25)
    q3 = np.percentile(DataWithoutNan,75)
    iqr=q3-q1
    upper_limit = q3 + 1.5 * iqr
    lower_limit = q1 - 1.5 * iqr
    return upper_limit, lower_limit

# preparing  bar plotting with one function
def paint_bar(df,title,x,y,x_name,y_name,):
    fig=px.bar(data_frame=df,x=x,y=y,text=y,)
    fig.update_yaxes(title=y_name)
    fig.update_xaxes(title=x_name)
    fig.update_layout(width=800,height=600)
    fig.update_traces(textposition='outside')
    fig.update_layout(title=title,title_x=.5,title_font_color="white",plot_bgcolor="black",paper_bgcolor="black",font_color="white",showlegend=False)
    fig.update_traces(marker_color="#F0C826")
    return fig
# styling the Plotly figure
def fig_style(fig,title=" ",legend=True):
    fig.update_layout(title=title,title_x=.5,title_font_color="white",plot_bgcolor="black",paper_bgcolor="black",font_color="white",showlegend=legend)
    fig.update_layout(width=800,height=600)
    fig.update_traces(marker_colors=color_list)
    return fig


def classifying_years(x):
    if float(x)==0:
        return "less than year" 
    elif float(x)>0 and float(x)<=5:
        return "1:5"
    elif float(x)>5 and float(x)<=10:
        return "6:10"
    elif float(x)>11 and float(x)<=15:
        return "11:15"  
    elif float(x)>15 :
        return "above 15 yrs"
    else : ## for NAN 
        return "Unkown"
    




In [5]:
schema.loc[schema["qname"].isin(questions_list)]

,qid,qname,question,force_resp,type,selector
3,QID312,Q120,,True,MC,SAVR
5,QID2,MainBranch,"Which of the following options best describes you today? For the purpose of this survey, a developer is ""someone who writes code"". *",True,MC,SAVR
6,QID127,Age,What is your age? *,True,MC,MAVR
7,QID296,Employment,Which of the following best describes your current employment status? Select all that apply.,False,MC,MAVR
8,QID308,RemoteWork,Which best describes your current work situation?,False,MC,SAVR
...,...,...,...,...,...,...
73,QID289,Knowledge_7,Waiting on answers to questions often causes interruptions and disrupts my workflow.,NaN,MC,MAVR
74,QID289,Knowledge_8,I feel like I have the tools and/or resources to quickly understand and work on any area of my company's code/system/platform.,NaN,MC,MAVR
75,QID290,Frequency_1,Needing help from people outside of your immediate team?,NaN,MC,MAVR
76,QID290,Frequency_2,Interacting with people outside of your immediate team?,NaN,MC,MAVR


##### Extracting the data professional data from the main Data file

In [6]:
mask=(df["DevType"].str.contains("data") | df["DevType"].str.contains("Data"))
mask.replace(np.nan,False,inplace=True)
data_df=df.loc[mask]

In [7]:
data_df.shape[0]

3930

In [8]:
data_df.isna().sum()

ResponseId                0
Q120                      0
MainBranch                0
Age                       0
Employment                0
                       ... 
ProfessionalTech       1853
Industry               2164
SurveyLength             47
SurveyEase               53
ConvertedCompYearly    1450
Length: 84, dtype: int64

In [9]:
tempdata=pd.DataFrame(mask.value_counts()).reset_index()
tempdata.at[0,"DevType"]="Other Professionals"
tempdata.at[1,"DevType"]="Data Professionals"
fig=px.pie(data_frame=tempdata,names="DevType",values="count",width=600,height=400)
fig_style(fig=fig,title="The percentage of Data Profesionals in the Survey",legend=True)
print("The count of Data professionals = ",data_df.shape[0])
print("The Percentage of Data professionals = ",(data_df.shape[0]/df.shape[0])*100)
fig.show()



C:\Users\W.D1998\AppData\Local\Temp\ipykernel_10184\3930809725.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Other Professionals' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  tempdata.at[0,"DevType"]="Other Professionals"


The count of Data professionals =  3930
The Percentage of Data professionals =  4.406620021528526


#### Some Changes to data professionals dataframe

In [10]:
mask=((data_df["MainBranch"]=="I am a developer by profession") | (data_df["MainBranch"]=="I am not primarily a developer, but I write code sometimes as part of my work/studies"))
data_df=data_df.loc[mask]

In [11]:
data_df["Industry"]=data_df["Industry"].apply(lambda x: "Information technology" if x=="Information Services, IT, Software Development, or other Technology" else (
    "Manufacturing and Transportation" if x=="Manufacturing, Transportation, or Supply Chain" else x))

In [12]:
#the job names is too long and ugly ,so lets clean it
data_df["DevType"]=data_df["DevType"].apply(lambda x : "Data scientist" if x =="Data scientist or machine learning specialist" 
                                            else ( "Data Engineer" if x=="Engineer, data" 
                                                  else ( "Data analyst"if x == "Data or business analyst" else x) ))

In [13]:
tempdata=pd.DataFrame(data_df['DevType'].value_counts()).reset_index()
print(tempdata)
fig=px.pie(data_frame=tempdata,names="DevType",values="count",width=600,height=400)
fig_style(fig=fig,title="The percentage of each Data Profesionals Catagory in the Survey",legend=True)

                  DevType  count
0          Data scientist   1537
1           Data Engineer   1217
2            Data analyst    703
3  Database administrator    229


### Which country have the highest number of data analysts ?

In [14]:
tempdata=data_df.Country.value_counts().head(10)

tempdata=pd.DataFrame(tempdata).reset_index()
tempdata.at[2,"Country"]="United Kingdom and Northern Ireland"

fig=paint_bar(title="Top 10 Countries having Data Professionals",df=tempdata,x="Country",y="count",x_name="Country",y_name="count")


#### which country have the highest number of data analysts ?

In [15]:
tempdata=pd.DataFrame((data_df[data_df["DevType"]=="Data analyst"]["Country"].value_counts())).reset_index()
tempdata=tempdata.head(10)
tempdata.at[1,"Country"]="United Kingdom and Northern Ireland"
paint_bar(df=tempdata,x="Country",y="count",x_name="Country",y_name="count",title="Top 10 Countries having Data Analaysts")

In [16]:
industry_df=count_column_content(data_df,"Industry")
# sns.barplot(data=industry_df,x="Industry",y="Count",color="white")
# plt.xticks(rotation=90)
# plt.xlabel("Count of data professionals")
# plt.ylabel("Data professionals")
# plt.title("Number of Data professionals per Industry")
paint_bar(df=industry_df,x="Industry",y="Count",title="Number of Data professionals per Industry",x_name="Industry",y_name="Data professionals Count")


In [17]:
fig=px.pie(data_frame=industry_df,values="Count",names="Industry")
fig=fig_style(fig=fig,title="Which industry have the hgier percentage of data professional",legend=True)
# fig.update_traces(marker_colors=color_list)
fig

In [18]:
industry_list=data_df.Industry.dropna().unique().tolist()
tempdata=pd.DataFrame(data_df.groupby(["Industry","DevType"])["ResponseId"].count()).reset_index()
tempdata=tempdata.loc[tempdata["Industry"]=="Healthcare"]
app=Dash(__name__)
app.layout=html.Div(id="main_div",children=[
    html.Div(id="drop_div",children=dcc.Dropdown(id="dropdown",value="Healthcare",options=industry_list)),
    html.Div(id="div_fig",children=[dcc.Graph(id="graph",figure=go.Figure())])
])
@callback(Output(component_id="graph",component_property="figure"),
          Input(component_id="dropdown",component_property="value"))
def grphing(val):
    tempdata=pd.DataFrame(data_df.groupby(["Industry","DevType"])["ResponseId"].count()).reset_index()
    tempdata=tempdata.loc[tempdata["Industry"]==f"{val}"]
    fig=px.pie(data_frame=tempdata,values="ResponseId",names="DevType")
    fig.update_traces(marker_colors=color_list)
    fig=fig_style(fig,title="Distrbution of Data professional roles inside selected industry",legend=True)
    return fig


if __name__ == "__main__":
    app.run(debug=True)



#### What are the most used programming language and databasess have used or desired by data profssionals?

In [19]:
used_pro_language=count_column_content(data_df,"LanguageHaveWorkedWith").head(10)
paint_bar(df=used_pro_language,title="Top 10 programming language used by Data professionals",
          x="LanguageHaveWorkedWith",y="Count",x_name="Language",y_name="Count")

In [20]:
pro_language_desired=count_column_content(data_df,"LanguageWantToWorkWith").head(10)
paint_bar(df=pro_language_desired,title="Top 10 programming language desired by data professionals",x="LanguageWantToWorkWith",y="Count",x_name="Programming language",y_name="Count")

In [21]:
database_used=count_column_content(data_df,"DatabaseHaveWorkedWith").head(10)
paint_bar(df=database_used,title="Top 10 Databases used by Data professionals",x="DatabaseHaveWorkedWith",y="Count",x_name="Database",y_name="Count")

In [22]:
database_desired=count_column_content(data_df,"DatabaseWantToWorkWith").head(10)
paint_bar(df=database_desired,title="Top 10 Databases desired by Data professionals",x="DatabaseWantToWorkWith",y="Count",x_name="Database",y_name="Count")

In [23]:
tempdata=count_column_content(data_df,'MiscTechHaveWorkedWith')
tempdata=tempdata.head(10)
paint_bar(df=tempdata,title="Top 10 Technologies used by Data Professionals",x="MiscTechHaveWorkedWith",y="Count",x_name="The Technology",y_name="Count")

In [24]:
tempdata=count_column_content(data_df,'MiscTechWantToWorkWith')
tempdata=tempdata.head(10)
paint_bar(df=tempdata,title="Top 10 Technologies desired by Data Professionals",x="MiscTechWantToWorkWith",y="Count",x_name="The Technology",y_name="Count")
tempdata

,MiscTechWantToWorkWith,Count
0,Pandas,1721
1,NumPy,1597
2,Torch/PyTorch,1299
3,Scikit-Learn,1297
4,TensorFlow,1004
5,Apache Spark,918
6,Apache Kafka,674
7,Hugging Face Transformers,667
8,Keras,599
9,Opencv,543


### which data professial have highest income

Exploring the outliers in "CompTotal" column

In [25]:
fig=px.box(data_df["CompTotal"])
fig.update_traces(marker_color="gold")
fig.update_layout(plot_bgcolor="black",paper_bgcolor="black")

In [26]:
upper_limit,lower_limit=get_upper_lower_limit(data_df,"CompTotal")
data_df["CompTotal"]=data_df["CompTotal"].apply(lambda x: upper_limit if x > upper_limit else (lower_limit if x < lower_limit else x) )

In [27]:
tempdata=pd.DataFrame(data_df.groupby("DevType")["CompTotal"].median()).reset_index()
paint_bar(df=tempdata,x="DevType",y="CompTotal",x_name="job Type",y_name="Median Salary",title="The Average income per job",)

In [28]:

data_df["YearsCodePro"]=data_df["YearsCodePro"].apply(cleaning_expiernce_years_of_writing_code)
data_df["YearsCodeProClasses"]=data_df["YearsCodePro"].apply(classifying_years)
data_df["WorkExpClasses"]=data_df["WorkExp"].apply(classifying_years)


### Exploring the Income


##### (when it comes to salary and income we need to discover for outliers)

In [29]:
############################ the percentgae of expiernce catagories ############################
coding_ex=pd.DataFrame(data_df.groupby("YearsCodeProClasses")["CompTotal"].count()).reset_index()
coding_ex.drop(index=3,inplace=True)


fig=px.pie(data_frame=coding_ex,values="CompTotal",names="YearsCodeProClasses")
fig=fig_style(fig,title="the Percentage of each (Coding) experince years catagory",legend=True)
fig.update_traces(marker_colors=color_list)


In [30]:
############################ the median salary per coding experinec ############################
coding_ex=pd.DataFrame(data_df.groupby("YearsCodeProClasses")["CompTotal"].median()).reset_index()
coding_ex.drop(index=3,inplace=True)   # deleting the unkown years experince


coding_ex.sort_values(by="CompTotal",inplace=True)
paint_bar(df=coding_ex,title="The Average salry per years of professional coding",x="YearsCodeProClasses",y="CompTotal",x_name="Experince Year Catagory",y_name="The median Salary")

In [31]:
Exp_class=pd.DataFrame(data_df.groupby("WorkExpClasses")["CompTotal"].count()).reset_index()
Exp_class.drop(index=3,inplace=True)


fig=px.pie(data_frame=Exp_class,values="CompTotal",names="WorkExpClasses")
fig=fig_style(fig,title="the Percentage of of each (Working) experince catagory",legend=True)

fig.update_traces(marker_colors=color_list)
fig

In [32]:
tempdata=pd.DataFrame(data_df.groupby("WorkExpClasses")["CompTotal"].median()).reset_index()
tempdata.drop(index=3,inplace=True)


tempdata.sort_values(by="CompTotal",inplace=True)
paint_bar(df=tempdata,title="The Average Salary per Working Experience of Data Professionals",x="WorkExpClasses",y="CompTotal",x_name="Experince Year Catagory",y_name="The median Salary")

In [33]:
fig=px.pie(data_frame=tempdata,values="CompTotal",names="WorkExpClasses")
fig_style(fig,title="the Percentage of of each (Working) experince years catagory",legend=True)

In [34]:
mask=~((data_df["Age"]=="Under 18 years old") | (data_df["Age"]=="Prefer not to say"))
temp=data_df.loc[mask]

temp=pd.DataFrame(temp.groupby("Age")["CompTotal"].median()).reset_index()
paint_bar(df=temp,title="The Average salary for each of these Age catagory",x="Age",y="CompTotal",x_name="Age Catagory",y_name="median Salary")


In [35]:
temp=pd.DataFrame(data_df.groupby("Industry")["CompTotal"].median()).reset_index()
temp.sort_values("CompTotal",inplace=True)
paint_bar(df=temp,x="Industry",y="CompTotal",title="The Average salary per industry",x_name="Industry",y_name="CompTotal")

In [36]:
ten=data_df["MiscTechHaveWorkedWith"].str.contains("TensorFlow")
tor=data_df["MiscTechHaveWorkedWith"].str.contains("Torch/PyTorch")

ten.fillna(False,inplace=True)
tor.fillna(False,inplace=True)
ten_df=data_df.loc[ten & ~tor]
tor_df=data_df.loc[~ten & tor]

tor_sample=tor_df["CompTotal"].dropna().sample(n=100,random_state=123456789)
ten_sample=ten_df["CompTotal"].dropna().sample(n=100,random_state=123456789)


C:\Users\W.D1998\AppData\Local\Temp\ipykernel_10184\3419126192.py:4: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\W.D1998\AppData\Local\Temp\ipykernel_10184\3419126192.py:5: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [37]:
tor_sample=pd.DataFrame(tor_sample).reset_index().drop(columns="index")
tor_sample["tech"]="PyTorch"
ten_sample=pd.DataFrame(ten_sample).reset_index().drop(columns="index")
ten_sample["tech"]="TensorFlow"


tempdata=pd.concat([ten_sample,tor_sample])
tempdata=tempdata.groupby("tech")["CompTotal"].median()
tempdata=tempdata.reset_index()
paint_bar(tempdata,x="tech",y="CompTotal",x_name="FrameWork",y_name="Salary",title="The Average income for FrameWork Users")

In [38]:
data_df.to_csv(path_or_buf=r"D:\C.V\stack overflow\data_df.csv",encoding="UTF-8")

### i can show off my skills more but it's enough 😂
### in the end i'm Client of myself 😂🤣